<a href="https://colab.research.google.com/github/kumpaten/masters-thesis-code/blob/main/DataOLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) Install required package
!pip install linearmodels

# 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# OLS Monolith

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson, jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import acorr_ljungbox
from linearmodels.panel import PanelOLS
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# --- LaTeX Style Plotting Setup ---
# (This should be adjusted based on your exact thesis LaTeX font and style if possible)
# For now, using common settings that look professional.
plt.style.use('seaborn-v0_8-whitegrid') # A clean style
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans'] # Or a list of preferred fallbacks
# Remove or comment out: plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 8
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['figure.figsize'] = (6, 4) # Adjust as needed for thesis layout (e.g., half-width)
plt.rcParams['lines.linewidth'] = 1.5
# For saving figures that LaTeX can use well:
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.format'] = 'pdf' # or 'png' or 'eps'

# 1) Load preprocessed dataset
# Make sure to upload your CSV to Colab or adjust the path
try:
    file_path = '/content/drive/MyDrive/Data_to_analyze/Final_Dataset_transformed.csv'
    df_original = pd.read_csv(file_path, sep=';')
except FileNotFoundError:
    print("ERROR: 'Final_Dataset_transformed.csv' not found. Please upload it or check the path.")
    # Fallback for testing if file is missing:
    # df_original = pd.DataFrame(np.random.rand(100, 15), columns=['company', 'year', 'ln(Tobin’s Q)', 'ln(PE)', 'ln(TSR)', 'brand_value', 'patent_claims', 'employee_smoothed_rating', 'R&D_Intensity', 'SG&A_Intensity', 'YJ(Sentiment_PCR)', 'ln_totalAssets-1', 'ln_totalSales-1', 'ROA-1', 'ln(Financial Leverage-1)', 'delta_ln_S5INFT-1', 'delta_ln_GDPWorld-1', 'is_imputed'])
    # df_original['company'] = [f'Comp{i%10}' for i in range(100)]
    # df_original['year'] = [2010 + i//10 for i in range(100)]


df_original['year'] = df_original['year'].astype(int)
df_original.set_index(['company', 'year'], inplace=True)

# 2) Define variable groups (ensure names exactly match CSV columns)
dependent_vars_list = ['ln(Tobin’s Q)', 'ln(PE)', 'ln(TSR)']

# Ensure these names are EXACTLY as in your CSV after any transformations you've done
contemporaneous_intangibles_base = [
    'Brand Value (USD m)', 'Patent Claims', 'Employee Rating', # Assuming these are the base names
    'R&D Intensity (%)', 'SG&A Intensity (%)', 'YJ(Sentiment_PCR)'
]
# Renaming to match typical Python variable naming conventions if needed, or ensure CSV matches
# For simplicity, I'll assume your CSV uses these or similar Python-friendly names:
contemporaneous_intangibles = [
    'brand_value', 'patent_claims', 'employee_smoothed_rating', # Changed from 'employee_smoothed_rating' for consistency if it's the main proxy
    'R&D_Intensity', 'SG&A_Intensity', 'YJ(Sentiment_PCR)' # Using underscores
]
# Check which set of names is correct for your 'Final_Dataset_transformed.csv'
# Let's assume Python-friendly names are in the CSV for now.
# If not, you'll need to rename columns in the DataFrame or use the original names here.
# Example: df.rename(columns={'Brand Value (USD m)': 'brand_value', ...}, inplace=True)
#'ln_totalSales-1'

firm_controls = [
    'ln_totalAssets-1', 'ROA-1', 'ln(Financial Leverage-1)' # Assuming Python-friendly names
]
macro_controls = [
    'delta_ln_S5INFT-1', 'delta_ln_GDPWorld-1' # Assuming Python-friendly names
]
dummy_controls = ["is_imputed"] # This should be 0 or 1

# --- Data Cleaning: Ensure column names match ---
# Create a mapping if your CSV column names are different
column_name_mapping = {
    'Brand Value (USD m)': 'brand_value',
    'Patent Claims': 'patent_claims',
    'Employee Rating': 'employee_smoothed_rating', # This was 'employee_smoothed_rating' in your code
    'R&D Intensity (%)': 'R&D_Intensity',
    'SG&A Intensity (%)': 'SG&A_Intensity',
    'YJ(Sentiment_PCR)': 'YJ(Sentiment_PCR)',
    'ln(Total Assets-1)': 'ln_totalAssets-1', # Example, adjust if your lag naming is different
    #'ln(Total Sales-1)': 'ln_totalSales-1', # ln(Total Sales-1) was dropped due to multicollinearity with ln(Total Assets-1)
    'ROA-1': 'ROA-1',
    'ln(Financial Leverage-1)': 'ln(Financial Leverage-1)',
    'delta_ln_S5INFT-1': 'delta_ln_S5INFT-1',
    'delta_ln_GDPWorld-1': 'delta_ln_GDPWorld-1',
    'is_imputed': 'is_imputed' # Assuming this is already Python-friendly
}
# Check if renaming is needed
df = df_original.copy()
cols_to_rename = {k: v for k, v in column_name_mapping.items() if k in df.columns}
if cols_to_rename:
    df.rename(columns=cols_to_rename, inplace=True)
    print(f"Renamed columns: {cols_to_rename}")

# Verify all defined variables exist in the DataFrame
all_defined_vars = dependent_vars_list + contemporaneous_intangibles + firm_controls + macro_controls + dummy_controls
missing_cols = [col for col in all_defined_vars if col not in df.columns and not col.endswith('_lag1')] # _lag1 will be created
if missing_cols:
    print(f"ERROR: The following defined variables are missing from the DataFrame: {missing_cols}")
    print(f"Available columns: {df.columns.tolist()}")
    # exit() # Or handle error appropriately

# 3) Create a working copy and generate lagged intangible variables
lagged_intangibles = []
for col in contemporaneous_intangibles:
    lagged_col_name = f'{col}_lag1'
    df[lagged_col_name] = df.groupby(level='company')[col].shift(1)
    lagged_intangibles.append(lagged_col_name)

# Also create lagged dummy control for Scenario 2
df['is_imputed_lag1'] = df.groupby(level='company')['is_imputed'].shift(1)
lagged_dummy_controls = ["is_imputed_lag1"]


# 4) Standardize ALL potential predictors that will be used in models
# We standardize after creating lagged versions to avoid data leakage from future to past via scaler fitting
# Standardize contemporaneous and lagged intangibles, firm controls, and macro controls
# Dummy variables are typically NOT standardized.
predictors_to_standardize = (
    contemporaneous_intangibles +
    lagged_intangibles +
    firm_controls +
    macro_controls
)

# Ensure no new NaNs were introduced by shift that are not handled by dropna() before scaling
df_for_scaler = df[predictors_to_standardize].copy()
# Important: Fit the scaler ONLY on data that is not NaN to avoid issues.
# Then transform the whole columns, NaNs will remain NaNs.
for col in predictors_to_standardize:
    if col in df_for_scaler.columns:
        col_data_no_na = df_for_scaler[col].dropna()
        if not col_data_no_na.empty:
            scaler = StandardScaler()
            # Fit scaler on non-NaN data
            df_for_scaler.loc[col_data_no_na.index, col] = scaler.fit_transform(col_data_no_na.values.reshape(-1, 1))
            # Apply transformation to the original dataframe column
            df[col] = scaler.transform(df[[col]].values.reshape(-1,1)) # Transform original df column
        else:
            print(f"Warning: Column '{col}' has no non-NaN data for scaler fitting. Will be all NaNs or not scaled.")
    else:
        print(f"Warning: Column '{col}' not found for standardization.")


# (Keep all imports and previous setup code the same)
# ...

# 5) MODIFIED Function to run models
def run_ols_scenario(
    scenario_label,
    dep_var_name,
    dataf,
    intangible_set,
    current_dummy_controls,
    run_ldv_on_model_c=False, # Renamed for clarity
    specific_models_to_run=None # New parameter: e.g., ['A', 'B', 'C'] or ['C']
):
    fitted_models_dict = {}
    print(f"\n{'='*60}")
    print(f" OLS SCENARIO: {scenario_label} ".center(60, "="))
    print(f" Dependent Variable: {dep_var_name} ".center(60, "="))
    intangible_type = 'Contemporaneous' if intangible_set == contemporaneous_intangibles else 'Lagged'
    print(f" Intangible Set: {intangible_type} ".center(60, "="))
    if run_ldv_on_model_c and (specific_models_to_run is None or 'C' in specific_models_to_run):
        print(" Model C will include Lagged Dependent Variable (LDV) ".center(60, "="))
    print(f"{'='*60}")

    # Define base predictors for this scenario
    base_model_a_predictors = list(intangible_set) + list(current_dummy_controls)
    base_model_b_predictors = base_model_a_predictors + firm_controls
    base_model_c_predictors = base_model_b_predictors + macro_controls

    # Full set of model specifications
    all_models_specs_dict = {
        "A": {"label": "Model A (Intangibles Only)", "predictors": base_model_a_predictors},
        "B": {"label": "Model B (+ Firm-Level Controls)", "predictors": base_model_b_predictors},
        "C": {"label": "Model C (Full Controls)", "predictors": base_model_c_predictors},
    }

    # Determine which models to actually run
    if specific_models_to_run: # If a specific list is provided
        models_to_process = {key: all_models_specs_dict[key] for key in specific_models_to_run if key in all_models_specs_dict}
    else: # Run all by default
        models_to_process = all_models_specs_dict

    current_df_scenario = dataf.copy()

    dep_var_lag1_name = f'{dep_var_name}_lag1_dv'
    # Prepare LDV only if Model C is being run and LDV is requested for it
    if 'C' in models_to_process and run_ldv_on_model_c:
        current_df_scenario[dep_var_lag1_name] = current_df_scenario.groupby(level='company')[dep_var_name].shift(1)
        ldv_data_no_na = current_df_scenario[dep_var_lag1_name].dropna()
        if not ldv_data_no_na.empty:
            scaler_ldv = StandardScaler()
            current_df_scenario.loc[ldv_data_no_na.index, dep_var_lag1_name] = scaler_ldv.fit_transform(ldv_data_no_na.values.reshape(-1, 1))
            print(f"  LDV '{dep_var_lag1_name}' prepared and standardized for Model C.")
        else:
            print(f"Warning: No data to fit scaler for LDV '{dep_var_lag1_name}'. LDV model might fail.")


    for model_key, spec in models_to_process.items():
        model_label_base = spec["label"]
        actual_predictors = list(spec["predictors"])
        display_model_label = model_label_base

        if model_key == "C" and run_ldv_on_model_c:
            if dep_var_lag1_name in current_df_scenario.columns:
                actual_predictors.append(dep_var_lag1_name)
                display_model_label = f"{model_label_base} + LDV"
            else:
                print(f"Warning: LDV '{dep_var_lag1_name}' not prepared for {model_label_base}. Running Model C without LDV.")

        print(f"\n--- Fitting {display_model_label} ---")

        missing_model_preds = [p for p in actual_predictors if p not in current_df_scenario.columns]
        if missing_model_preds:
            print(f"ERROR: Predictors missing for {display_model_label}: {missing_model_preds}. Skipping.")
            continue

        model_data = current_df_scenario[[dep_var_name] + actual_predictors].dropna()

        if model_data.empty or len(model_data) < (len(actual_predictors) + model_data.index.get_level_values('company').nunique() + 5) :
            print(f"No/Insufficient data for {display_model_label} (Obs: {len(model_data)}, Preds: {len(actual_predictors)}). Skipping.")
            continue

        y_model = model_data[dep_var_name]
        X_model_df = model_data[actual_predictors]

        try:
            mod = PanelOLS(y_model, sm.add_constant(X_model_df, has_constant='add'), entity_effects=True, drop_absorbed=True)
            model_fit = mod.fit(cov_type='clustered', cluster_entity=True)

            fitted_models_dict[display_model_label] = model_fit
            print(model_fit.summary)

            if model_key == "C" and run_ldv_on_model_c:
                print("  Note: Coefficient on Lagged DV may be subject to Nickell bias.")

            # Re-estimate Model A with i.i.d. errors (only if Model A is run and it's not an LDV focused run for Model C)
            if model_key == "A" and not run_ldv_on_model_c :
                 print(f"\n--- Re-estimating {model_label_base} with Homoskedastic (i.i.d.) Errors for Comparison ---")
                 mod_iid = PanelOLS(y_model, sm.add_constant(X_model_df, has_constant='add'), entity_effects=True, drop_absorbed=True)
                 model_fit_iid = mod_iid.fit()
                 print(model_fit_iid.summary)

            # --- Diagnostics ---
            print(f"\n--- Diagnostics for {display_model_label} ({intangible_type} Intangibles) ---")
            residuals = model_fit.resids.copy()
            exog_for_bp_test = sm.add_constant(X_model_df.loc[residuals.index], has_constant='add').values

            if len(residuals) > exog_for_bp_test.shape[1]:
                try:
                    bp_test_results = het_breuschpagan(residuals, exog_for_bp_test)
                    print(f"Breusch-Pagan Test: LM Stat={bp_test_results[0]:.3f}, LM p-val={bp_test_results[1]:.3f}, F Stat={bp_test_results[2]:.3f}, F p-val={bp_test_results[3]:.3f}")
                    if bp_test_results[1] < 0.05 or bp_test_results[3] < 0.05: print("  Evidence of heteroskedasticity (p < 0.05). Clustered SEs appropriate.")
                    else: print("  No strong evidence of heteroskedasticity (p >= 0.05).")
                except Exception as e: print(f"  Could not run Breusch-Pagan test: {e}")
            else: print("  Not enough observations for Breusch-Pagan test after handling NaNs for exog.")

            if len(residuals) > 1:
                dw_stat = durbin_watson(residuals)
                print(f"Durbin-Watson Statistic: {dw_stat:.3f}")
                if dw_stat < 1.5: print("  Indicates strong positive serial correlation.")
                elif dw_stat > 2.5: print("  Indicates strong negative serial correlation.")
                elif 1.5 <= dw_stat < 1.9: print("  Indicates some positive serial correlation.")
                elif 2.1 < dw_stat <= 2.5: print("  Indicates some negative serial correlation.")
                else: print("  DW in inconclusive/no strong serial correlation range (around 2).")
            if len(residuals) > 20:
                try:
                    n_lags = min(10, len(residuals)//5 -1 )
                    if n_lags > 0 :
                        lb_results = acorr_ljungbox(residuals, lags=n_lags, return_df=True)
                        print("Ljung-Box Q Test (up to {} lags):".format(n_lags))
                        print(lb_results)
                        if (lb_results['lb_pvalue'] < 0.05).any(): print("  Evidence of serial correlation (Ljung-Box). Clustered SEs appropriate.")
                        else: print("  No strong evidence of serial correlation (Ljung-Box).")
                    else: print("  Not enough observations for meaningful Ljung-Box test lags.")
                except Exception as e: print(f"  Could not run Ljung-Box test: {e}")
            else: print("  Not enough residuals for Ljung-Box test.")

            if len(residuals) > 2:
                jb_stat, jb_pvalue, skew, kurtosis = jarque_bera(residuals)
                print(f"Jarque-Bera Test: Stat={jb_stat:.3f}, p-val={jb_pvalue:.3f}, Skew={skew:.3f}, Kurtosis={kurtosis:.3f}")
                if jb_pvalue < 0.05: print("  Residuals may not be normally distributed (p < 0.05).")
                else: print("  Residuals appear normally distributed (p >= 0.05).")

                plt.figure()
                stats.probplot(residuals, dist="norm", plot=plt)
                plot_title = f'Q-Q Plot: {dep_var_name}\n{display_model_label} ({intangible_type} Intangibles)'
                plt.title(plot_title, fontsize=10) # Adjusted title for brevity
                plt.xlabel("Theoretical Quantiles (Normal Distribution)")
                plt.ylabel("Sample Quantiles (Residuals)")
                filename_label = display_model_label.replace(" ", "_").replace("(", "").replace(")", "").replace("+", "plus").replace("/", "or")
                dv_filename_part = dep_var_name.replace("(", "").replace(")", "").replace("’", "").replace(" ", "_")
                plot_filename = f"qq_plot_{dv_filename_part}_{scenario_label}_{filename_label}.pdf"
                plt.savefig(plot_filename, bbox_inches='tight')
                plt.show()
                print(f"  Saved Q-Q plot to: {plot_filename}")
            else: print("  Not enough residuals for Jarque-Bera test or Q-Q plot.")

            print("\nVariance Inflation Factors (VIF):")
            X_for_vif_calc = X_model_df.dropna()
            if X_for_vif_calc.shape[1] > 1 and len(X_for_vif_calc) > X_for_vif_calc.shape[1]:
                try:
                    vif_data = pd.DataFrame()
                    vif_data["feature"] = X_for_vif_calc.columns
                    vif_data["VIF"] = [variance_inflation_factor(X_for_vif_calc.values, i) for i in range(X_for_vif_calc.shape[1])]
                    print(vif_data.sort_values('VIF', ascending=False))
                    if (vif_data["VIF"] > 10).any(): print("  Warning: Potential multicollinearity detected (VIF > 10 for some features).")
                    elif (vif_data["VIF"] > 5).any(): print("  Note: Some multicollinearity present (VIF > 5 for some features).")
                    else: print("  No strong evidence of multicollinearity (all VIFs <= 5).")
                except Exception as e: print(f"  Could not calculate VIF: {e} (Shape: {X_for_vif_calc.shape})")
            elif X_for_vif_calc.shape[1] <=1 : print("  Not enough predictors for VIF (need >1).")
            else: print(f"  Not enough observations after NaN drop for VIF (Obs: {len(X_for_vif_calc)}, Preds: {X_for_vif_calc.shape[1]}).")

        except Exception as e:
            print(f"Could not fit or run diagnostics for {display_model_label}: {e}")
            continue

    return fitted_models_dict

# 6) Execute OLS Analysis as per Methodology

# Scenario 1: Contemporaneous Intangibles (Models A, B, C)
print("\n\n" + "*"*80)
print(" EXECUTING OLS SCENARIO 1: CONTEMPORANEOUS INTANGIBLES (Models A, B, C) ".center(80, "*"))
print("*"*80 + "\n")
all_fitted_models_scenario1 = {}
for dv in dependent_vars_list:
    all_fitted_models_scenario1[dv] = run_ols_scenario(
        scenario_label="Scenario1_Contemp",
        dep_var_name=dv,
        dataf=df.copy(),
        intangible_set=contemporaneous_intangibles,
        current_dummy_controls=dummy_controls,
        run_ldv_on_model_c=False, # Model C here does NOT include LDV
        specific_models_to_run=['A', 'B', 'C'] # Explicitly run all three
    )

# Scenario 1 - Model C with LDV (Robustness Check - ONLY RUN MODEL C)
print("\n\n" + "*"*80)
print(" EXECUTING OLS SCENARIO 1: MODEL C with LDV (ROBUSTNESS) ".center(80, "*"))
print("*"*80 + "\n")
all_fitted_models_scenario1_model_c_ldv = {} # Renamed for clarity
for dv in dependent_vars_list:
    all_fitted_models_scenario1_model_c_ldv[dv] = run_ols_scenario(
        scenario_label="Scenario1_Contemp_ModelC_LDV", # More specific label
        dep_var_name=dv,
        dataf=df.copy(),
        intangible_set=contemporaneous_intangibles,
        current_dummy_controls=dummy_controls,
        run_ldv_on_model_c=True, # This will augment Model C with LDV
        specific_models_to_run=['C'] # KEY CHANGE: Only run Model C
    )

# Scenario 2: Lagged Intangibles (Models A, B, C)
print("\n\n" + "*"*80)
print(" EXECUTING OLS SCENARIO 2: LAGGED INTANGIBLES (Models A, B, C) ".center(80, "*"))
print("*"*80 + "\n")
all_fitted_models_scenario2 = {}
for dv in dependent_vars_list:
    all_fitted_models_scenario2[dv] = run_ols_scenario(
        scenario_label="Scenario2_Lagged",
        dep_var_name=dv,
        dataf=df.copy(),
        intangible_set=lagged_intangibles,
        current_dummy_controls=lagged_dummy_controls,
        run_ldv_on_model_c=False,
        specific_models_to_run=['A', 'B', 'C'] # Explicitly run all three
    )

# Scenario 2 - Model C with LDV (Robustness Check - ONLY RUN MODEL C)
print("\n\n" + "*"*80)
print(" EXECUTING OLS SCENARIO 2: MODEL C with LDV (ROBUSTNESS) ".center(80, "*"))
print("*"*80 + "\n")
all_fitted_models_scenario2_model_c_ldv = {} # Renamed for clarity
for dv in dependent_vars_list:
    all_fitted_models_scenario2_model_c_ldv[dv] = run_ols_scenario(
        scenario_label="Scenario2_Lagged_ModelC_LDV", # More specific label
        dep_var_name=dv,
        dataf=df.copy(),
        intangible_set=lagged_intangibles,
        current_dummy_controls=lagged_dummy_controls,
        run_ldv_on_model_c=True,
        specific_models_to_run=['C'] # KEY CHANGE: Only run Model C
    )

print("\n\n" + "="*30 + " END OF OLS ANALYSIS " + "="*30)